In [ ]:
import re
from bs4 import BeautifulSoup
import unicodedata
import configparser
ini = configparser.ConfigParser()
ini.read('./conf.ini')

from selenium import webdriver
driver = webdriver.Chrome('/usr/local/bin/chromedriver')

import time
#인스타그램 접속
driver.get('https://www.instagram.com/')
time.sleep(2)

email = ini['instagram']['instagram_id']
pw = ini['instagram']['instagram_pw']

input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(pw)
input_pw.submit()
time.sleep(10)

def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url


def select_first(driver):
    first = driver.find_element_by_css_selector("div._aabd._aa8k._aanf")
    first.click()
    time.sleep(3)

def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        content = soup.select('div._a9zs > span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '

    tags = re.findall(r'#[^\s#,\\]+', content)

    date = soup.select('time._a9ze._a9zf')[0]['datetime'][:10]

    try:
        like = soup.select('div._aacl._aaco._aacw._aacx._aada._aade > span')[0].text
    except:
        like = 0

    try:
        place = soup.select('div._aaqm')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    data = [content, date, like, place, tags]
    return data

def move_next(driver):
    right = driver.find_elements('css selector', 'div._aaqg._aaqh > button._abl-')[0]
    right.click()
    time.sleep(1.5)

word = '제주도맛집'
url = insta_searching(word)
driver.get(url)
time.sleep(5)

select_first(driver)

results = []

target = 10
for i in range(target):
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)

print(results[:2])
